In [1]:
import sys
sys.path.append('../')

In [2]:
from tank import Tank, Hole, create_holes
import ipywidgets as widgets
from ipycanvas import canvas
from IPython.display import display, Markdown, Latex
import time
from demo import *
from pythreejs import *

In [3]:
c = canvas.Canvas(width=600, height=250)
m = Tank(create_holes(25, 2), 0.06, c=c)

scene = Scene(children=[m.tank_rendering, m.water_rendering, camera, AmbientLight(color='#777777')], background=None)

renderer = Renderer(camera=camera,
                    scene=scene,
                    alpha=True,
                    clearOpacity=0,
                    controls=[OrbitControls(controlling=camera)],
                    width=600, height=200)

demo = Demo(m.params, m)

/home/kipoka/anaconda3/envs/hydraulics/lib/python3.10/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [4]:
demo.show()
c
renderer

Output()

Output()

Renderer(camera=PerspectiveCamera(aspect=3.0, children=(DirectionalLight(color='white', intensity=0.5, positio…